# Week 7 Assignment

_MkKinney 6.1_

This week has been all about getting information off the internet both in structured data formats (CSV, JSON, etc) as well as HTML.  For these exercises, we're going to use two practical examples of fetching data from web pages to show how to use Pandas and BeautifulSoup to extract structured information from the web.

---
---

### 33.1 Parsing a list in HTML

Go to the Banner Health Price Transparency Page: https://www.bannerhealth.com/patients/billing/pricing-resources/hospital-price-transparency

Notice that there is a list of hospitals and the city they are in.  We want to parse the underlying HTML to create a list of all the hospitals along with which city they're in.

```json
[
    ["Banner - University Medical Center Phoenix", "Arizona"],
    ["Banner - University Medical Center South ", "Arizona"],
    ...
]
```

To examine the underlying HTML code, you can use Chrome, right-click, and choose **Inspect**.

For reference, the documentation for BeautifulSoup is here: https://www.crummy.com/software/BeautifulSoup/bs4/doc/


In the cell below, create a function called **parse_banner(url)** that takes as it's one parameter the URL of the webpage to be parsed for links.  Make sure you include docstrings and a good test case using hte URL provided above.

In [1]:
from bs4 import BeautifulSoup
import requests

# Note that you'll need to fetch the data using the following syntax to include headers
# that make the web server think you're a real web browser.

url = 'https://www.bannerhealth.com/patients/billing/pricing-resources/hospital-price-transparency'
headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36" }
response = requests.get(url, headers=headers)

In [2]:
response.status_code

200

In [3]:
price_transparency = BeautifulSoup(response.text)

In [4]:
def parse_banner(url):
    """(str) - > (list)
    This function takes the url of the site, reads the undelying HTML and returns the list of lists containing states and hospitals.
    
    >>> len(parse_banner('https://www.bannerhealth.com/patients/billing/pricing-resources/hospital-price-transparency'))
    32
    
    >>> element = parse_banner(url)
    >>> element[0]
    ['Banner - University Medical Center Phoenix', 'Arizona']
    
    >>> element[4][0]
    'Banner Behavioral Health Hospital'
    
    """
   
    ## finding all the paragraphs in the HTML and assigning it to allPara
    
    allPara = price_transparency.find_all('p')
    
    ## creating empty lists and dictionary
    
    list2 = []
    dictionary = {}
    
    ## looping through all the paragraphs and selscting only paragraphs containing states. Then assigning them in the dictionary to the empty list
    
    for states in allPara:
        for state in states.find_all('strong'):
            if state.text not in dictionary:
                dictionary[state.text] = []
    
    ## finding all the next siblings.
            list1= []
        
            for hospitals in states.findNextSiblings():
     
    ## breaking the loop if the next sibling is p, else running it to find the li tags
                
                if hospitals.name == 'p':
                    break
                if hospitals.name == 'ul':
                    for hospital in hospitals.find_all('li'):
                
    ## appending the text of li to list1 if it is not there in the dictionary keys
                        
                        if hospital.text not in dictionary.keys():
                            list1.append(hospital.text.replace("\xa0", ""))

            dictionary[state.text] = list1

    ## for every key and values in the dictionary appending them in list2 
            
    for key in dictionary:
        values = dictionary[key]
        for value in values:
            list2.append([value,key])
#     print(list2)
    return list2

In [5]:
import doctest
doctest.run_docstring_examples(parse_banner, globals(), verbose=True)

Finding tests in NoName
Trying:
    len(parse_banner('https://www.bannerhealth.com/patients/billing/pricing-resources/hospital-price-transparency'))
Expecting:
    32
ok
Trying:
    element = parse_banner(url)
Expecting nothing
ok
Trying:
    element[0]
Expecting:
    ['Banner - University Medical Center Phoenix', 'Arizona']
ok
Trying:
    element[4][0]
Expecting:
    'Banner Behavioral Health Hospital'
ok


In [6]:
banner = parse_banner('https://www.bannerhealth.com/patients/billing/pricing-resources/hospital-price-transparency')
assert len(banner)==32, 'Length of result should have been 38, but {} returned.'.format(len(banner))
assert banner[0][1]=='Arizona', 'Wrong data found in the first result item: {}'.format(banner[0])

---

## 33.2 Using a REST API (from GitHub.com)

Many websites provide something called a REST API to access information from their site programatically, rather than relying on HTML.  One example is GitHub.com, whose API allows you do to things like "list all the public repositories for a user."

The documentation for GitHub.com's REST API can be found here: https://docs.github.com/en/rest/guides/getting-started-with-the-rest-api

Create a function called **repo_summary(user)** that takes a GitHub.com user name as it's parameter and retrieves a list of all the repositories you can see for that user.  The specific documentation for the this kind of request can be found here: https://docs.github.com/en/rest/reference/repos#list-repositories-for-a-user. Make sure your function is well documented with a docstring and includes a simple test to verify that you get back 12 repositories when querying for the repositories for user **paulboal**.

I've provided a related example to help you out.

In [7]:
# Example -- this example of code shows how to get basic information on the user paulboal
# For your solution, make sure you meet the requirements in the instructions above.

import requests

response = requests.get('https://api.github.com/users/paulboal')
data = response.json()
print('This information is about {}. His website is {}.'.format(data.get('login'), data.get('blog')))

This information is about paulboal. His website is www.amitechsolutions.com.


In [8]:
def repo_summary(user):
    """('str') -> ['repository names']
    This function takes the username and searches the total repositories of that user. Then append them into a list.
    
    >>> len(repo_summary('paulboal'))
    12
    """
    ## creating a empty list called repository
    
    repository = []
    
    ## appending the input user to the url and assigning it to variable called request
    
    request = requests.get('https://api.github.com/users/'+user+'/repos')
    json = request.json()    
    
    ## looping through the entire json file to find repository name, and appending it to the empty list called repository
    
    for i in range(0,len(json)):
        repository_name = (json[i]['name'])
        repository.append(repository_name)
    return repository


In [9]:
import doctest
doctest.run_docstring_examples(repo_summary, globals(), verbose=True)

Finding tests in NoName
Trying:
    len(repo_summary('paulboal'))
Expecting:
    12
ok


In [10]:
repos = repo_summary('paulboal')
assert len(repos)==12, 'Expecing 12, but {} were found'.format(len(repos))

---

### 33.3 Find Something of Your Own

Do some web searches and find an HTML page with some data that is interesting to something you're studying.  You can extract and parse that information using either BeautifulSoup or Pandas.  If you're using Pandas, then do something interesting to format and structure your data.  If you're using BeautifulSoup, you'll just need to do the work of parsing the data out of HTML -- that's hard enough!

You don't need to build this as a function.  Just use notebook cells as I've done above.  You will be graded based on _style_.  Use variable names that make sense for your problem / solution. Cleanup anything you don't need before you submit your work.

In [11]:
# Your Code Here

In [12]:
## importing pandas 

import pandas as pd

## reading the html of wikipedia page 'science' and assigning it to the variable called table

table = pd.read_html('https://en.wikipedia.org/wiki/Science')

## selecting the required table i.e., table 2 and assigning it to research_impactcs to know the important researches and their impact on science  

research_impacts = table[2]
research_impacts

,Research,Impact
0,Static electricity and magnetism (c. 1600)Elec...,"All electric appliances, dynamos, electric pow..."
1,Diffraction (1665),"Optics, hence fiber optic cable (1840s), moder..."
2,Germ theory (1700),"Hygiene, leading to decreased transmission of ..."
3,Vaccination (1798),Leading to the elimination of most infectious ...
4,Photovoltaic effect (1839),"Solar cells (1883), hence solar power, solar p..."
5,The strange orbit of Mercury (1859) and other ...,"Satellite-based technology such as GPS (1973),..."
6,Radio waves (1887),Radio had become used in innumerable ways beyo...
7,Radioactivity (1896) and antimatter (1932),"Cancer treatment (1896), Radiometric dating (1..."
8,X-rays (1896),"Medical imaging, including computed tomography."
9,Crystallography and quantum mechanics (1900),"Semiconductor devices (1906), hence modern com..."


In [13]:
## printing the column names in the form of a list

list(research_impacts.columns)

['Research', 'Impact']

In [14]:
## styling the dataframe using style.set_properties and giving the dataframe a background colour, border, and color

research_impacts.style.set_properties(**{'background-color': 'pink',
                               'border': '1.3px solid grey',   
                                'color': 'purple'})

,Research,Impact
0,Static electricity and magnetism (c. 1600)Electric current (18th century),"All electric appliances, dynamos, electric power stations, modern electronics, including electric lighting, television, electric heating, transcranial magnetic stimulation, deep brain stimulation, magnetic tape, loudspeaker, and the compass and lightning rod."
1,Diffraction (1665),"Optics, hence fiber optic cable (1840s), modern intercontinental communications, and cable TV and internet."
2,Germ theory (1700),"Hygiene, leading to decreased transmission of infectious diseases; antibodies, leading to techniques for disease diagnosis and targeted anticancer therapies."
3,Vaccination (1798),Leading to the elimination of most infectious diseases from developed countries and the worldwide eradication of smallpox.
4,Photovoltaic effect (1839),"Solar cells (1883), hence solar power, solar powered watches, calculators and other devices."
5,The strange orbit of Mercury (1859) and other researchleading to special (1905) and general relativity (1916),"Satellite-based technology such as GPS (1973), satnav and satellite communications.[l]"
6,Radio waves (1887),"Radio had become used in innumerable ways beyond its better-known areas of telephony, and broadcast television (1927) and radio (1906) entertainment. Other uses included – emergency services, radar (navigation and weather prediction), medicine, astronomy, wireless communications, geophysics, and networking. Radio waves also led researchers to adjacent frequencies such as microwaves, used worldwide for heating and cooking food."
7,Radioactivity (1896) and antimatter (1932),"Cancer treatment (1896), Radiometric dating (1905), nuclear reactors (1942) and weapons (1945), mineral exploration, PET scans (1961), and medical research (via isotopic labeling)."
8,X-rays (1896),"Medical imaging, including computed tomography."
9,Crystallography and quantum mechanics (1900),"Semiconductor devices (1906), hence modern computing and telecommunications including the integration with wireless devices: the mobile phone,[l] LED lamps and lasers."


In [15]:
## styling the dataframe using style.set_properties and giving the dataframe a background colour, border, and color

research_impacts.style.set_properties(**{'border': '1.3px solid grey',
                                'color': 'blue'})

,Research,Impact
0,Static electricity and magnetism (c. 1600)Electric current (18th century),"All electric appliances, dynamos, electric power stations, modern electronics, including electric lighting, television, electric heating, transcranial magnetic stimulation, deep brain stimulation, magnetic tape, loudspeaker, and the compass and lightning rod."
1,Diffraction (1665),"Optics, hence fiber optic cable (1840s), modern intercontinental communications, and cable TV and internet."
2,Germ theory (1700),"Hygiene, leading to decreased transmission of infectious diseases; antibodies, leading to techniques for disease diagnosis and targeted anticancer therapies."
3,Vaccination (1798),Leading to the elimination of most infectious diseases from developed countries and the worldwide eradication of smallpox.
4,Photovoltaic effect (1839),"Solar cells (1883), hence solar power, solar powered watches, calculators and other devices."
5,The strange orbit of Mercury (1859) and other researchleading to special (1905) and general relativity (1916),"Satellite-based technology such as GPS (1973), satnav and satellite communications.[l]"
6,Radio waves (1887),"Radio had become used in innumerable ways beyond its better-known areas of telephony, and broadcast television (1927) and radio (1906) entertainment. Other uses included – emergency services, radar (navigation and weather prediction), medicine, astronomy, wireless communications, geophysics, and networking. Radio waves also led researchers to adjacent frequencies such as microwaves, used worldwide for heating and cooking food."
7,Radioactivity (1896) and antimatter (1932),"Cancer treatment (1896), Radiometric dating (1905), nuclear reactors (1942) and weapons (1945), mineral exploration, PET scans (1961), and medical research (via isotopic labeling)."
8,X-rays (1896),"Medical imaging, including computed tomography."
9,Crystallography and quantum mechanics (1900),"Semiconductor devices (1906), hence modern computing and telecommunications including the integration with wireless devices: the mobile phone,[l] LED lamps and lasers."


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [ ]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add week07_assignment_2.ipynb
    !git commit -a -m "Submitting the week 7 programming exercises"
    !git push
else:
    print('''
    
OK. We can wait.
''')